In [26]:
# libraries
import os
import csv
import time
import tensorflow as tf
import bert
from bert import run_classifier
import pandas as pd
from NYT_parser import NYTArticle
from utilities import *

W0706 21:08:38.151100 140372267288384 deprecation_wrapper.py:119] From /home/mark/anaconda3/lib/python3.7/site-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [2]:
# get pathways
train_paths = data_setup.get_paths('train')
article = NYTArticle.from_file(os.path.join('data', train_paths[0][0]))

In [3]:
# set destination folder and make folders if needed
dest_folder = 'transdata/train/'
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

In [4]:
# attempt a single file output
print("Writing", dest_folder+article.docid)
article.simple_csv_output(dest_folder)

Writing transdata/train/739212


In [9]:
# attempt 50 article write minus filtered out
for path in train_paths[:50]:
    article = NYTArticle.from_file(os.path.join('data',path[0]))
    if article.pass_filters():
        print("Writing", dest_folder+article.docid)
        article.simple_csv_output(dest_folder)

Writing transdata/train/1397928
Writing transdata/train/1609775
Writing transdata/train/297100
Writing transdata/train/448007
Writing transdata/train/1463610
Writing transdata/train/590096
Writing transdata/train/762936
Writing transdata/train/1570877
Writing transdata/train/1725268
Writing transdata/train/1647864
Writing transdata/train/1251662
Writing transdata/train/551924
Writing transdata/train/74723
Writing transdata/train/1535917
Writing transdata/train/744457
Writing transdata/train/97129
Writing transdata/train/899133
Writing transdata/train/791668
Writing transdata/train/779448
Writing transdata/train/1287650
Writing transdata/train/927546
Writing transdata/train/1587613
Writing transdata/train/638654
Writing transdata/train/870914
Writing transdata/train/800527
Writing transdata/train/1584905
Writing transdata/train/1287373
Writing transdata/train/1337862
Writing transdata/train/1646102
Writing transdata/train/1363692
Writing transdata/train/50986
Writing transdata/train/148

In [11]:
# write all filtered articles in train set
opened_article = 0
written_article = 0
start = time.time()
for path in train_paths:
    opened_article += 1
    article = NYTArticle.from_file(os.path.join('data',path[0]))
    if article.pass_filters():
        written_article += 1
        article.simple_csv_output(dest_folder)
    if opened_article % 10000 == 0:
        print("Opened", opened_article,"articles. Output", written_article,"articles.")
print(f"\n... processed {opened_article} files in {time.time() - start} seconds")


... processed 1001 files in 2.489995241165161 seconds


In [17]:
# build dataframe with articles
sample = 1000

headlines = []
bodies = []

for path in train_paths[:sample]:
    article = NYTArticle.from_file(os.path.join('data',path[0]))
    if article.pass_filters():
        article.simple_csv_output(dest_folder)
        headlines.append(article.print_hede[0])
        # join paragraphs for now, may want to do something else later
        bodies.append('\n'.join(article.paragraphs))

train_df = pd.DataFrame(data={'headline':headlines, 'body':bodies})
train_df.head()

,headline,body
0,"A Killer, Yes, but She's a Good Old Girl",Aileen Wuornos was big news 10 years ago. At a...
1,"How High's The Bidding, Mama?",THE blue Rolls Royce sat at the York Street cu...
2,Pacificorp reports earnings for Qtr to Sept 30,LEAD:\n*3*** COMPANY REPORTS **\n*3* Pacificor...
3,Stewart Set to Return,There is bad news for Oakland's competition in...
4,Taxing Online Sales,"To the Editor:\nRe ''So Many Online Sales, So ..."


## Trying to imitate BERT

In [27]:
DATA_COLUMN = 'body'
LABEL_COLUMN = 'headline'

# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train_df.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

# test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
#                                                                    text_a = x[DATA_COLUMN], 
#                                                                    text_b = None, 
#                                                                    label = x[LABEL_COLUMN]),